# Análise de Dados, Aplicada ao Estudo de Diabetes Entre Indígenas Pima, Utilizando Python, SQL e Banco de Dados Relacional.

Os indígenas Pima são uma comunidade indígena que habitam na região desértica do sudoeste dos Estados Unidos da América e no norte do México. Nos EUA eles têm uma das taxas mais altas de diabetes do mundo. A análise e classificação de obesos, pela definição da OMS, entre as indígenas Pima com mais de 35 anos é importante para identificar tendências e padrões.

## Instalando e Importando Pacotes.

In [1]:
# Instala o pacote iPython-sql. Utilizado para executar SQL no Jupyter Notebbok.
# https://pypi.org/project/ipython-sql/
%pip install -q ipython-sql

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Carrega o pacote iPython-sql como um comando mágico no Jupyter Notebook.
%load_ext sql

In [3]:
# Instala o pacote watermark. Utilizado para obter informações sobre as versões de outros pacotes utilizados neste Jupyter Notebbok.
# https://pypi.org/project/watermark/
%pip install -q watermark

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Carrega o pacote watermark como um comando mágico no Jupyter Notebook.
%load_ext watermark

In [5]:
# Importa pacotes necessários para esse projeto.
import contextlib
import sqlite3
import pandas as pd

from os import remove

In [6]:
# Versão do Python e de todos os pacotes importados neste projeto.
%watermark -a "Samuel G. Ribeiro" -gu "sgribeiro" --python --iversions

Author: Samuel G. Ribeiro

Github username: sgribeiro

Python implementation: CPython
Python version       : 3.9.11
IPython version      : 7.21.0

pandas : 1.3.5
sqlite3: 2.6.0



## Carregando o conjunto de dados

In [7]:
#Carrega o dataset como um Dataframe do pandas.
df = pd.read_csv('dataset/diabetes.csv')

## Analise exploratória dos dados com Python

In [8]:
# Quantidade de linhas e colunas do conjunto de dados.
rows, columns = df.shape
print(f"Linhas: {rows}\nColunas: {columns}")

Linhas: 768
Colunas: 9


In [9]:
#Colunas.
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [10]:
# Amostra dos dados. Primeiras cinco linhas do conjunto de dados.
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [11]:
# Visualiza os tipos de dados.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


## Conecção do Banco de Dados Relacional com Python

In [12]:
# Apaga o banco de dados caso ele exista na pasta do projeto.
with contextlib.suppress(FileNotFoundError):
    remove('database/diabetes-pima.db')

In [13]:
# Conecta ao banco de dados SQLite. Se não existir ele é criado.
connection = sqlite3.connect('database/diabetes-pima.db')

In [14]:
# Insere o dataframe 'diabetes' no banco de dados como uma tabela.
df.to_sql('diabetes', connection)

In [15]:
# Define o banco de dados.
%sql sqlite:///database/diabetes-pima.db

## Manipulação de Dados com SQL

In [16]:
%%sql

-- Conta quantos registros há na tabela diabetes.
SELECT count(*)
FROM diabetes

 * sqlite:///database/diabetes-pima.db
Done.


count(*)
768


In [17]:
%%sql

-- Criando uma nova tabela de pacientes.
CREATE TABLE patients (Pregnancies INT,
                       Glucose INT,
                       BloodPressure INT,
                       SkinThickness INT,
                       Insulin INT,
                       BMI DECIMAL(8, 2),
                       DiabetesPedigreeFunction DECIMAL(8, 2),
                       Age INT,
                       Outcome INT);

 * sqlite:///database/diabetes-pima.db
Done.


[]

In [18]:
%%sql

-- Insere na nova tabela todas as pacientes com mais de 35 anos.
INSERT INTO patients(Pregnancies, 
                     Glucose, 
                     BloodPressure, 
                     SkinThickness, 
                     Insulin, 
                     BMI, 
                     DiabetesPedigreeFunction, 
                     Age, 
                     Outcome) 
SELECT Pregnancies, 
       Glucose, 
       BloodPressure, 
       SkinThickness, 
       Insulin, 
       BMI, 
       DiabetesPedigreeFunction, 
       Age, 
       Outcome 
FROM diabetes 
WHERE Age > 35;

 * sqlite:///database/diabetes-pima.db
Done.


[]

In [19]:
%%sql

-- Conta quantos registros há na tabela patients.
SELECT count(*)
FROM patients

 * sqlite:///database/diabetes-pima.db
Done.


count(*)
270


In [20]:
%%sql

-- Seleciona todos os dados da nova tabela de pacientes.
SELECT * 
FROM patients

 * sqlite:///database/diabetes-pima.db
Done.


Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
6,148,72,35,0,33.6,0.627,50,1
2,197,70,45,543,30.5,0.158,53,1
8,125,96,0,0,0,0.232,54,1
10,139,80,0,0,27.1,1.441,57,0
1,189,60,23,846,30.1,0.398,59,1
5,166,72,19,175,25.8,0.587,51,1
8,99,84,0,0,35.4,0.388,50,0
7,196,90,0,0,39.8,0.451,41,1
11,143,94,33,146,36.6,0.254,51,1
10,125,70,26,115,31.1,0.205,41,1


In [21]:
%%sql 

-- Cria uma nova coluna na tabela de pacientes.
ALTER TABLE patients
ADD WeightStatus VARCHAR(10);

 * sqlite:///database/diabetes-pima.db
Done.


[]

A coluna criada será preenchida seguindo a classificação de obesidade da Organização Mundial da Saúde.
<table border = "1">
    <tr>
        <td colspan = "2">WHO Classification</td>
    </tr>
    <tr>
        <td>Weight Status</td>
        <td>BMI (kg/m²)</td>
    </tr>
    <tr>
        <td>Underweight</td>
        <td>< 18.5</td>
    </tr>
    <tr>
        <td>Normal range</td>
        <td>18.5 - 24.9</td>
    </tr>
        <tr>
        <td>Pre-obese</td>
        <td>25 - 29.9</td>
    </tr>
        <tr>
        <td>Obese I</td>
        <td>30 - 34.9</td>
    </tr>
        <tr>
        <td>Obese II</td>
        <td>35 - 39.9</td>
    </tr>
        <tr>
        <td>Obese III</td>
        <td>>= 40</td>
    </tr>
</table>

In [22]:
%%sql

-- Seleciona todos os dados da nova tabela de pacientes para visualizar a nova coluna criada.
SELECT * 
FROM patients

 * sqlite:///database/diabetes-pima.db
Done.


Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,WeightStatus
6,148,72,35,0,33.6,0.627,50,1,None
2,197,70,45,543,30.5,0.158,53,1,None
8,125,96,0,0,0,0.232,54,1,None
10,139,80,0,0,27.1,1.441,57,0,None
1,189,60,23,846,30.1,0.398,59,1,None
5,166,72,19,175,25.8,0.587,51,1,None
8,99,84,0,0,35.4,0.388,50,0,None
7,196,90,0,0,39.8,0.451,41,1,None
11,143,94,33,146,36.6,0.254,51,1,None
10,125,70,26,115,31.1,0.205,41,1,None


In [23]:
%%sql

-- Categoriza os pacientes como 'Underweight' com base no atributo BMI.
UPDATE patients
SET WeightStatus  = 'Underweight'
WHERE BMI < 18.5

 * sqlite:///database/diabetes-pima.db
Done.


[]

In [24]:
%%sql

-- Seleciona todos os dados da nova tabela de pacientes para visualizar a categoria 'Underweight'.
SELECT * 
FROM patients

 * sqlite:///database/diabetes-pima.db
Done.


Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,WeightStatus
6,148,72,35,0,33.6,0.627,50,1,None
2,197,70,45,543,30.5,0.158,53,1,None
8,125,96,0,0,0,0.232,54,1,Underweight
10,139,80,0,0,27.1,1.441,57,0,None
1,189,60,23,846,30.1,0.398,59,1,None
5,166,72,19,175,25.8,0.587,51,1,None
8,99,84,0,0,35.4,0.388,50,0,None
7,196,90,0,0,39.8,0.451,41,1,None
11,143,94,33,146,36.6,0.254,51,1,None
10,125,70,26,115,31.1,0.205,41,1,None


In [25]:
%%sql

-- Categoriza os pacientes como 'Normal' com base no atributo BMI.
UPDATE patients
SET WeightStatus = 'Normal range'
WHERE BMI >= 18.5 
AND BMI <= 24.9;

 * sqlite:///database/diabetes-pima.db
Done.


[]

In [26]:
%%sql

-- Seleciona todos os dados da nova tabela de pacientes para visualizar a categoria 'Normal range'.
SELECT * 
FROM patients

 * sqlite:///database/diabetes-pima.db
Done.


Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,WeightStatus
6,148,72,35,0,33.6,0.627,50,1,None
2,197,70,45,543,30.5,0.158,53,1,None
8,125,96,0,0,0,0.232,54,1,Underweight
10,139,80,0,0,27.1,1.441,57,0,None
1,189,60,23,846,30.1,0.398,59,1,None
5,166,72,19,175,25.8,0.587,51,1,None
8,99,84,0,0,35.4,0.388,50,0,None
7,196,90,0,0,39.8,0.451,41,1,None
11,143,94,33,146,36.6,0.254,51,1,None
10,125,70,26,115,31.1,0.205,41,1,None


In [27]:
%%sql

-- Categoriza os pacientes como 'Pre-obese' com base no atributo BMI.
UPDATE patients
SET WeightStatus  = 'Pre-obese'
WHERE BMI >= 25 
AND BMI <= 29.9;

 * sqlite:///database/diabetes-pima.db
Done.


[]

In [28]:
%%sql

-- Seleciona todos os dados da nova tabela de pacientes para visualizar a categoria 'Pre-obese'.
SELECT * 
FROM patients

 * sqlite:///database/diabetes-pima.db
Done.


Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,WeightStatus
6,148,72,35,0,33.6,0.627,50,1,None
2,197,70,45,543,30.5,0.158,53,1,None
8,125,96,0,0,0,0.232,54,1,Underweight
10,139,80,0,0,27.1,1.441,57,0,Pre-obese
1,189,60,23,846,30.1,0.398,59,1,None
5,166,72,19,175,25.8,0.587,51,1,Pre-obese
8,99,84,0,0,35.4,0.388,50,0,None
7,196,90,0,0,39.8,0.451,41,1,None
11,143,94,33,146,36.6,0.254,51,1,None
10,125,70,26,115,31.1,0.205,41,1,None


In [29]:
%%sql

-- Categoriza os pacientes como 'Obese I' com base no atributo BMI.
UPDATE patients
SET WeightStatus = 'Obese I'
WHERE BMI >= 30 
AND BMI <= 34.9;

 * sqlite:///database/diabetes-pima.db
Done.


[]

In [30]:
%%sql

-- Seleciona todos os dados da nova tabela de pacientes para visualizar a categoria 'Obese I'.
SELECT * 
FROM patients

 * sqlite:///database/diabetes-pima.db
Done.


Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,WeightStatus
6,148,72,35,0,33.6,0.627,50,1,Obese I
2,197,70,45,543,30.5,0.158,53,1,Obese I
8,125,96,0,0,0,0.232,54,1,Underweight
10,139,80,0,0,27.1,1.441,57,0,Pre-obese
1,189,60,23,846,30.1,0.398,59,1,Obese I
5,166,72,19,175,25.8,0.587,51,1,Pre-obese
8,99,84,0,0,35.4,0.388,50,0,None
7,196,90,0,0,39.8,0.451,41,1,None
11,143,94,33,146,36.6,0.254,51,1,None
10,125,70,26,115,31.1,0.205,41,1,Obese I


In [31]:
%%sql

-- Categoriza os pacientes como 'Obese II' com base no atributo BMI.
UPDATE patients
SET WeightStatus = 'Obese II'
WHERE BMI >= 35 
AND BMI <= 39.9;

 * sqlite:///database/diabetes-pima.db
Done.


[]

In [32]:
%%sql

-- Seleciona todos os dados da nova tabela de pacientes para visualizar a categoria 'Obese II'.
SELECT * 
FROM patients

 * sqlite:///database/diabetes-pima.db
Done.


Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,WeightStatus
6,148,72,35,0,33.6,0.627,50,1,Obese I
2,197,70,45,543,30.5,0.158,53,1,Obese I
8,125,96,0,0,0,0.232,54,1,Underweight
10,139,80,0,0,27.1,1.441,57,0,Pre-obese
1,189,60,23,846,30.1,0.398,59,1,Obese I
5,166,72,19,175,25.8,0.587,51,1,Pre-obese
8,99,84,0,0,35.4,0.388,50,0,Obese II
7,196,90,0,0,39.8,0.451,41,1,Obese II
11,143,94,33,146,36.6,0.254,51,1,Obese II
10,125,70,26,115,31.1,0.205,41,1,Obese I


In [33]:
%%sql

-- Categoriza os pacientes como 'Obese III' com base no atributo BMI.
UPDATE patients
SET WeightStatus = 'Obese III'
WHERE BMI >= 40;

 * sqlite:///database/diabetes-pima.db
Done.


[]

In [34]:
%%sql

-- Seleciona todos os dados da nova tabela de pacientes para visualizar a categorização completa.
SELECT * 
FROM patients

 * sqlite:///database/diabetes-pima.db
Done.


Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,WeightStatus
6,148,72,35,0,33.6,0.627,50,1,Obese I
2,197,70,45,543,30.5,0.158,53,1,Obese I
8,125,96,0,0,0,0.232,54,1,Underweight
10,139,80,0,0,27.1,1.441,57,0,Pre-obese
1,189,60,23,846,30.1,0.398,59,1,Obese I
5,166,72,19,175,25.8,0.587,51,1,Pre-obese
8,99,84,0,0,35.4,0.388,50,0,Obese II
7,196,90,0,0,39.8,0.451,41,1,Obese II
11,143,94,33,146,36.6,0.254,51,1,Obese II
10,125,70,26,115,31.1,0.205,41,1,Obese I


In [45]:
%%sql

-- Quantidade de cada status.
SELECT DISTINCT(WeightStatus), COUNT(WeightStatus) AS Count
FROM patients
GROUP BY WeightStatus

 * sqlite:///database/diabetes-pima.db
Done.


WeightStatus,Count
Normal range,27
Obese I,90
Obese II,63
Obese III,31
Pre-obese,57
Underweight,2


## Exportando os resultados como um arquivo .csv com Python

In [36]:
# Query.
query = connection.execute("SELECT * FROM patients")
query

In [37]:
# Colunas Atualizadas.
update_columns = [column[0] for column in query.description]
update_columns

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome',
 'WeightStatus']

In [38]:
# Gerando o Dataframe.
result = pd.DataFrame.from_records(data = query.fetchall(), columns = update_columns)

In [39]:
# Quantidade de linhas e colunas do conjunto de dados.
rows, columns = result.shape
print(f"Linhas: {rows}\nColunas: {columns}")

Linhas: 270
Colunas: 10


In [40]:
# Amostra dos dados. Primeiras cinco linhas do conjunto de dados atualizado.
result.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,WeightStatus
0,6,148,72,35,0,33.6,0.627,50,1,Obese I
1,2,197,70,45,543,30.5,0.158,53,1,Obese I
2,8,125,96,0,0,0.0,0.232,54,1,Underweight
3,10,139,80,0,0,27.1,1.441,57,0,Pre-obese
4,1,189,60,23,846,30.1,0.398,59,1,Obese I


In [41]:
# Salva em como um .csv
result.to_csv('dataset/patients.csv', index = False)